In [ ]:
pip install pandas openpyxl

In [ ]:
import pandas as pd
from google.colab import files
import io

def processar_pagamentos_colab():
    print("--- Sistema de Conferência (Com Validação de Não Cadastrados - CPF) ---")

    # 1. Upload
    print("\n Passo 1: Selecione os arquivos 'Ativos Banco' e 'Ativos Empresa'.")
    uploaded = files.upload()

    # Identificação dos arquivos
    nome_arq_banco = ""
    nome_arq_empresa = ""

    for nome_arquivo in uploaded.keys():
        if "banco" in nome_arquivo.lower():
            nome_arq_banco = nome_arquivo
        elif "empresa" in nome_arquivo.lower():
            nome_arq_empresa = nome_arquivo

    if not nome_arq_banco or not nome_arq_empresa:
        print("\nERRO: Não identifiquei os arquivos automaticamente.")
        return

    # 2. Data
    data_competencia = input("\n Passo 2: Insira a Data de Competência: ")

    try:
        print("\n Passo 3: Processando...")

        # --- ALTERAÇÃO PRINCIPAL NA LEITURA DO ARQUIVO DO BANCO ---
        # Leitura Banco (CPF na Coluna C/Index 2, Matrícula na Coluna D/Index 3)
        # Assumindo que o CPF é a chave para a Matrícula.

        df_banco = pd.read_excel(io.BytesIO(uploaded[nome_arq_banco]))

        # Criação do dicionário: Chave = CPF (Index 2), Valor = Matrícula (Index 3)
        # Importante: O CPF deve ser formatado (string, sem espaços) para o matching.
        dicionario_banco = dict(zip(
            df_banco.iloc[:, 2].astype(str).str.strip(), # Coluna 2 (Index C) para CPF
            df_banco.iloc[:, 3]                           # Coluna 3 (Index D) para Matrícula
        ))

        # Leitura Empresa
        df_empresa = pd.read_excel(io.BytesIO(uploaded[nome_arq_empresa]))

        lista_final = []

        # Conferência
        for index, linha in df_empresa.iterrows():
            try:
                # Dados da Empresa
                nome_empresa_original = str(linha.iloc[1]) # Coluna B

                # --- NOVO PONTO DE FOCO: CPF DA EMPRESA ---
                # Pega o CPF da empresa na Coluna D (Index 3) e formata para matching.
                cpf_empresa_original = str(linha.iloc[3])
                cpf_empresa_formatado = cpf_empresa_original.strip() # Apenas 'strip' é suficiente para CPFs

                valor_empresa = linha.iloc[7] # Coluna H

                # LÓGICA DE VERIFICAÇÃO POR CPF
                if cpf_empresa_formatado in dicionario_banco:
                    # CASO 1: Encontrou o CPF no Banco
                    matricula = dicionario_banco[cpf_empresa_formatado]
                else:
                    # CASO 2: NÃO Encontrou o CPF no Banco
                    matricula = "Não Cadastrado no Banco"

                # Adiciona na lista (funciona para os dois casos)
                lista_final.append({
                    "Nome do Beneficiário": nome_empresa_original,
                    "CPF": cpf_empresa_original,
                    "Valor": valor_empresa,
                    "Data de Competência": data_competencia,
                    "Matrícula": matricula # Aqui entra o número ou o aviso
                })

            except Exception as e:
                # Omitir ou logar erros de processamento de linha específica.
                # print(f"Erro ao processar linha {index}: {e}")
                continue

        # Resultado
        if lista_final:
            nome_saida = "Planilha_Pagamento_Final_Por_CPF.xlsx"
            pd.DataFrame(lista_final).to_excel(nome_saida, index=False)

            print(f"\n SUCESSO! {len(lista_final)} registros processados.")
            files.download(nome_saida)
        else:
            print("\n A lista final ficou vazia.")

    except Exception as e:
        print(f"Erro: {e}")

if __name__ == "__main__":
    processar_pagamentos_colab()